In [1]:
#@Initialize and prepare Whisper engironment

%pip install -q git+https://github.com/openai/whisper.git
%pip install -q python-docx
!git clone -b master https://github.com/bhnatt/whisper-am.git

#https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
fatal: destination path 'whisper-am' already exists and is not an empty directory.


In [3]:
#@input parameters
import sys
sys.path.append('whisper-am')

from whisper_am import WhisperAM, mountDrive
from whisper_utils import write_txt, write_srt

data_dir = 'data/'

ds = mountDrive ()
google_drive = '/content/drive/MyDrive/' if ds else './'

data_path   = google_drive + data_dir   #@param {type:"string"}
print (data_path)

model_name = 'tiny.en' #@param ["medium.en", "large", "tiny.en"]

am = WhisperAM (model_name, data_path)
am.checkCuda ()

In [4]:
%%time
#@title run Whisper
import time

print ('Start :', time.strftime('%X %x %Z'))
am.run ()
print ('End  :', time.strftime('%X %x %Z'))

Start : 04:52:37 12/02/22 GMT
['./data/dream.mp3']
dream
dream 
  I have the pleasure to present to you, Dr. Martin ... d Negro spiritual and yet last, thank God Almighty
End  : 04:53:09 12/02/22 GMT
CPU times: user 41.7 s, sys: 3.34 s, total: 45 s
Wall time: 31.4 s
